# Imports

In [19]:
!pip install gspread
!pip install oauth2client
!pip install slackclient
!pip install -U snowflake-connector-python
!pip install -U snowflake-snowpark-python
!pip install --upgrade psycopg2
!pip install -U sqlalchemy
!pip install df2gspread

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [20]:
from snowflake.snowpark import Session 
import os
import boto3
import base64
import json
from requests import get
from pathlib import Path
from datetime import datetime, timedelta, date
import time
import zipfile
import io
import sqlalchemy
from pathlib import Path
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import pandas as pd
import numpy as np
import psycopg2
import sys
import requests
from botocore.exceptions import ClientError
import importlib
from io import StringIO
import snowflake.connector
from sqlalchemy import create_engine
import asyncio
import nest_asyncio
import re
import pytz
import pandas as pd
import snowflake.connector

# Defining Functions

In [21]:
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
from requests import get
from pathlib import Path
from datetime import datetime

def imports():
        import os
        import pandas as pd
        import numpy as np
        import psycopg2

import json
import boto3
import os
from pathlib import Path

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])
        
def initialize_env():
    snowflake_sg_secret = json.loads(get_secret("Snowflake-sagemaker"))
    dwh_writer_secret = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))
    metabase_secret = json.loads(get_secret("prod/metabase/maxab_config"))

    os.environ["SNOWFLAKE_USERNAME"] = snowflake_sg_secret["username"]
    os.environ["SNOWFLAKE_PASSWORD"] = snowflake_sg_secret["password"]
    os.environ["SNOWFLAKE_ACCOUNT"] = snowflake_sg_secret["account"]
    os.environ["SNOWFLAKE_DATABASE"] = snowflake_sg_secret["morocco_database"]

    os.environ["DWH_WRITER_HOST_NEW"] = dwh_writer_secret["host"]
    os.environ["DWH_WRITER_NAME_NEW"] = dwh_writer_secret["dbname"]
    os.environ["DWH_WRITER_USER_NAME_NEW"] = dwh_writer_secret["username"]
    os.environ["DWH_WRITER_PASSWORD_NEW"] = dwh_writer_secret["password"]
    
    os.environ["AFRICA_METABASE_USERNAME"] = metabase_secret["metabase_user"]
    os.environ["AFRICA_METABASE_PASSWORD"] = metabase_secret["metabase_password"]
    os.environ["AFRICA_METABASE_URL"] = metabase_secret["metabase_morocco_site"]

    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets
    

def get_from_gsheet(workbook, sheet):
    scope = ["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
    initialize_env()
    creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
    client = gspread.authorize(creds)
    try:
        wks = client.open(workbook).worksheet(sheet)
        sheet = pd.DataFrame(wks.get_all_records())
    except:
        print(sheet,'failed to fetch data')
    
    return sheet

In [22]:
import os
import requests
import json
import pandas as pd
import numpy as np
import re
import datetime
import seaborn as sns
from io import StringIO, BytesIO
import sys
from datetime import datetime

def ret_metabase(base_url, username, password, card_id):
    card_id = str(card_id)

    initialize_env()

    base_url = base_url + 'api'
    base_headers = {'Content-Type': 'application/json'}

    try:
        s_response = requests.post(
            base_url + '/session',
            data=json.dumps({
                'username': username,
                'password': password
            }),
            headers=base_headers)

        s_response.raise_for_status()

        session_token = s_response.json()['id']
        base_headers['X-Metabase-Session'] = session_token

        p_response = requests.post(
            base_url + '/card/' + card_id + '/query/xlsx',
            headers=base_headers
        )
        p_response.raise_for_status()

        my_dict = p_response.content
        xlsx_data = BytesIO(my_dict)
        df = pd.read_excel(xlsx_data, engine='openpyxl')
        df.columns = df.columns.str.lower()
        return df
    except Exception as e:
        raise e
    
#run query save on metabse with ID, 
#for example here query 1606 is save in my personal collection and I can run it like line below
# ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],9707)

In [23]:
import pytz
from datetime import datetime

initialize_env()

# Time zone for Morocco
morocco_tz = pytz.timezone('Africa/Casablanca')

# Get current time in Moroccan time zone
now = datetime.now(morocco_tz)
current_time = now.time()
current_day = now.strftime('%A')
print(current_day)
hour = int(str(now.time())[0:2])
print(now)
# print(current_day)
print(hour)

#Agents sheet
scope = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
client = gspread.authorize(creds)

sheet = client.open('MA daily_automatic_assignment')
agents_sheet_instance = sheet.worksheet('attendance_filtered')
agents_df = pd.DataFrame(agents_sheet_instance.get())
agents_df.columns = agents_df.iloc[0]
agents_df = agents_df[1:]
agents_df.reset_index(drop=True, inplace=True)
# print(agents_df)
# print(agents_df.columns)

# agents_df
# print(hour)
#Agents Assignment 
agents_df_copy = agents_df.copy()
agents_df_copy['assignment_start_time'] = agents_df['start_time'].astype(int)-1
agents_df_copy['assignment_end_time'] = agents_df['end_time'].astype(int)
agents_df_copy['day'] = agents_df['Day']

agents_df_copy['assign_data'] =  np.where(
    (hour >= agents_df_copy['assignment_start_time'])
    & (hour <= agents_df_copy['assignment_end_time'])
    & (agents_df_copy['day']=='TRUE'),
    'yes',
    'no' )

# print(agents_df_copy)
available_agents = agents_df_copy.loc[agents_df_copy['assign_data'] =='yes']
available_agents_list = available_agents['agent_id'].values.tolist()
number_of_available_agents = len(available_agents_list)
print(available_agents_list)

# # Separate agents based on the 'Project' column
# acquisition_agents = available_agents[available_agents['Project'] == 'acquisition']
# retention_agents = available_agents[available_agents['Project'] == 'retention']

# acquisition_agents_list = acquisition_agents['agent_id'].values.tolist()
# retention_agents_list = retention_agents['agent_id'].values.tolist()



Saturday
2025-05-24 09:37:35.958773+01:00
9
['128', '142', '120', '129', '141', '138', '144']


In [17]:
sheet = client.open('Telesales Cycle Assignment')
cycle_assignment_instance = sheet.worksheet('Sheet1')
cycle_assignment_df = pd.DataFrame(cycle_assignment_instance.get())
cycle_assignment_df.columns = cycle_assignment_df.iloc[0]
cycle_assignment_df = cycle_assignment_df[1:]
cycle_assignment_df.reset_index(drop=True, inplace=True)
print(cycle_assignment_df)
print(cycle_assignment_df.columns)
cycle_assignment_df = cycle_assignment_df[['MOBILE', 'THREE_CX_ID']]
print(cycle_assignment_df)
print(cycle_assignment_df.columns)

0        SEGMENT RETAILER_ID  THP     MOBILE AGENT_ID THREE_CX_ID
0           dead      67,076    0  674382999    6,017         117
1           dead      68,809    0  625071892    6,017         117
2           dead      25,122    0  705352615    6,017         117
3           dead      54,224    0  638096326    6,017         117
4           dead      59,427    0  606629204    6,017         117
...          ...         ...  ...        ...      ...         ...
1863  below 1000      72,336   40  714777311    6,022         144
1864  below 1000      62,628  270  610639872    6,022         144
1865  below 1000      63,860  710  612128349    6,022         144
1866  below 1000      66,561   60  661901395    6,022         144
1867  below 1000      25,954  220  682204736    6,022         144

[1868 rows x 6 columns]
Index(['SEGMENT', 'RETAILER_ID', 'THP', 'MOBILE', 'AGENT_ID', 'THREE_CX_ID'], dtype='object', name=0)
0        MOBILE THREE_CX_ID
0     674382999         117
1     625071892         1

In [25]:
def assign_data(df, list):
    import pandas as pd
    import numpy as np

    actual_length = len(df) % len(list)
    rows_needed = len(list) - actual_length
    columns_needed = len(df.axes[1])
    print(rows_needed)
    print(columns_needed)
    df = df.sample(frac=1)
    empty_df = pd.DataFrame(index=np.arange(rows_needed), columns=np.arange(columns_needed))
    assgn = pd.concat([df, empty_df])
    assgn = assgn.dropna(axis=1, how='all')
    mplr = len(assgn) / len(list) 
    assgn = assgn.assign(agent_assigned=[val for val in list for _ in range(int(mplr))])
    assgn = assgn.dropna()

    if assgn.shape[1] != 1:
        assgn["retailer_id"] = assgn["retailer_id"].fillna('').astype(str).str.replace(".0","",regex=False)
        assgn["mobile"] = assgn["mobile"].fillna('').astype(str).str.replace(".0","",regex=False)
        return assgn
    else:
        assgn.drop(['agent_assigned'], axis=1)
        assgn = assgn.iloc[0:0]
        assgn = assgn.assign(index="", retailer_id = "",name = "", mobile="",sales_order_id="",created_at="",project_name="",description="",agent_assigned="",last_call_date="",balance="")
        assgn = assgn[["index","retailer_id","name","mobile","district","sales_order_id","created_at","last_recharge_date","project_name","description","agent_assigned","last_call_date","balance"]]
        return assgn

In [24]:
!pip install gspread_dataframe


In [27]:

from common_functions import ret_metabase, google_sheets, send_text_slack
from datetime import datetime, timedelta          

import pytz
from datetime import datetime
import pandas as pd
import numpy as np

morocco_tz = pytz.timezone('Africa/Casablanca')
now = datetime.now(morocco_tz)
current_time = now.time()
current_day = now.strftime('%A')
print(current_day)
hour = int(str(now.time())[0:2])



agents_df = google_sheets('MA daily_automatic_assignment', 'attendance_filtered', 'get')
# print(agents_df)
print(hour)

agents_df_copy = agents_df.copy() 
agents_df.columns = agents_df.columns.str.strip().str.lower()
agents_df_copy['assignment_start_time'] = agents_df['start_time'].astype(int)
agents_df_copy['assignment_end_time'] = agents_df['end_time'].astype(int)
agents_df_copy['current_day'] = agents_df['day'].astype(str).str.lower() == 'true'

# Fetch data for mapped retailer-agent 
mapping_df = google_sheets('MA Telesales Cycle Assignment', 'Sheet1', 'get')


agents_df_copy['assign_data'] =  np.where(
    (hour >= agents_df_copy['assignment_start_time'])
    & (hour <= agents_df_copy['assignment_end_time'])
    & (agents_df['day'] == True),
    'yes',
    'no' )

# store all available clocked-in agents for today
available_agents = agents_df_copy.loc[agents_df_copy['assign_data'] == 'yes']
available_agents_list = available_agents['agent_id'].values.tolist()
number_of_available_agents = len(available_agents_list)

if number_of_available_agents != 0:
    
    if hour < 12:
    # Recharge data for all agents
        df_recharge = ret_metabase('MOROCCO', 10368)
        df_recharge.columns = map(str.lower, df_recharge.columns)
        data_assigned = assign_data(df_recharge, available_agents_list)


    else:
        df_recharge = ret_metabase('MOROCCO', 10368)
        df_recharge.columns = map(str.lower, df_recharge.columns)
        data_assigned = assign_data(df_recharge, available_agents_list)


    # Concatenate the dataframes along the rows (union all)
    df_filtered = data_assigned.drop_duplicates(subset=['retailer_id'], keep="first")
    df_filtered['retailer_id'] = df_filtered['retailer_id'].replace(',', '', regex=True).astype('int')
    df_filtered = df_filtered.reset_index(drop=True)
    # print(df_filtered)

    #Remove the data that already exists if found 
    data_instance = google_sheets('Fintech Runsheet', 'Data', 'get')
    data_instance_df = data_instance.drop(columns=['Unnamed: 0'])
    data_instance_df.columns = data_instance_df.iloc[0]
    data_instance_df = data_instance_df[1:].reset_index(drop=True)
    data_instance_df = data_instance_df.dropna(subset=['Retailer ID']).reset_index(drop=True)

    if(data_instance_df.empty==False):
        main_data_to_assign = pd.DataFrame(df_filtered.loc[~df_filtered['retailer_id'].isin(data_instance_df['Retailer ID'].astype(int).values)])
        main_data_assigned = assign_data(main_data_to_assign, available_agents_list)
    else:
        main_data_to_assign = df_filtered.copy()
        main_data_assigned = assign_data(main_data_to_assign, available_agents_list)
        # print(main_data_assigned)

    #Dispatching data to agents' sheet 
    final_data_to_assign = main_data_assigned.copy()
    final_data_to_assign['added_at'] = now.strftime('%Y-%m-%d %H:%M')

    if final_data_to_assign.columns[0] == 'index':
        final_data_to_assign = final_data_to_assign.drop(labels='index', axis=1)
    pass

    final_data_to_assign = final_data_to_assign.astype(str)

    google_sheets('Fintech Runsheet', 'raw_data', 'append', df=final_data_to_assign)

    file_name = f'''{now.strftime("%Y")}/{now.strftime("%m")}/{now.strftime("%d")}/account_mgmt_dispatched_data'''

    try:
        hist_data_from_s3 = pd.read_excel(f'''s3://snowflake-maxabdwh-morocco/dwh-initial-load/fintech/account_mgmt_dispatching/{file_name}.xlsx''')
        hist_data_from_s3 = pd.concat([hist_data_from_s3, final_data_to_assign], ignore_index=True)
    except Exception as e:
        print(e)
        hist_data_from_s3 = final_data_to_assign.copy()

    hist_data_from_s3.to_excel(
        f'''s3://snowflake-maxabdwh-morocco/dwh-initial-load/fintech/account_mgmt_dispatching/{file_name}.xlsx''',
        index=False
    )
else:
    print("no agents available")

Saturday
/home/ec2-user/service_account_key.json
9
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
4
10
/home/ec2-user/service_account_key.json
4
10
/home/ec2-user/service_account_key.json


# Functions

In [1]:
## assigning data in any project
def assign_data(df, list):

    actual_length = len(df) % len(list)
    rows_needed = len(list) - actual_length
    columns_needed = len(df.axes[1])
    print(rows_needed)
    print(columns_needed)
    df = df.sample(frac=1)
    empty_df = pd.DataFrame(index=np.arange(rows_needed), columns=np.arange(columns_needed))
    assgn = pd.concat([df, empty_df])
    assgn = assgn.dropna(axis=1, how='all')
    mplr = len(assgn) / len(list) 
    assgn = assgn.assign(agent_assigned=[val for val in list for _ in range(int(mplr))])
    assgn = assgn.dropna()
    # print(assgn)
    if assgn.shape[1] != 1:

            assgn["retailer_id"] = assgn["retailer_id"].fillna('').astype(str).str.replace(".0","",regex=False)
            assgn["mobile"] = assgn["mobile"].fillna('').astype(str).str.replace(".0","",regex=False)
            # df = assgn.groupby('agent_assigned').head(assigns)
            return assgn
    else:
            assgn.drop(['agent_assigned'], axis=1)
            assgn = assgn.iloc[0:0]
            assgn = assgn.assign(index="", retailer_id = "",name = "", mobile="",sales_order_id="",created_at="",updated_at="",total_order_price="",agent_assigned="")
            assgn = assgn[["index","retailer_id","name","mobile","sales_order_id","created_at","updated_at","total_order_price","agent_assigned"]]
            # df = assgn.groupby('agent_assigned').head(assigns)
            return assgn

In [22]:
import pytz
from datetime import datetime

initialize_env()

# Time zone for Morocco
morocco_tz = pytz.timezone('Africa/Casablanca')

# Get current time in Moroccan time zone
now = datetime.now(morocco_tz)
current_time = now.time()
current_day = now.strftime('%A')
print(current_day)
hour = int(str(now.time())[0:2])
# print(now)
# print(current_day)
# print(hour)

#Agents sheet
scope = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
client = gspread.authorize(creds)

sheet = client.open('[HOURLY] Fintech Runsheet')
agents_sheet_instance = sheet.worksheet('agents_sheet')
agents_df = pd.DataFrame(agents_sheet_instance.get())
agents_df.columns = agents_df.iloc[0]
agents_df = agents_df[1:]
agents_df.reset_index(drop=True, inplace=True)
print(agents_df)
# agents_df
print(hour)
#Agents Assignment 
agents_df_copy = agents_df.copy()
agents_df_copy['assignment_start_time'] = agents_df['start_time'].astype(int)
agents_df_copy['assignment_end_time'] = agents_df['end_time'].astype(int)
agents_df_copy['agents_days_off'] = agents_df['days_off'].str.split(', ')

agents_df_copy['assign_data'] =  np.where((hour >= agents_df_copy['assignment_start_time'] )
                                &
                                (hour <= agents_df_copy['assignment_end_time'])
                                & 
                                (str(current_day) not in str(agents_df_copy['agents_days_off']) )
                                , 'yes','no' )

# agents_df_copy['assign_data'] =  np.where(agents_df['agent_id']==108
#                                 , 'yes','no' )

print(agents_df_copy)
available_agents = agents_df_copy.loc[agents_df_copy['assign_data'] =='yes']
available_agents_list = available_agents['agent_id'].values.tolist()
number_of_available_agents = len(available_agents_list)
# print(number_of_available_agents)


if number_of_available_agents != 0:
    
    #en route query 
    # df_1 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10000)
    
    #created and delivery tomorrow query 
    df_1 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10085)
    
    #data cleaning 
    df_1.columns = map(str.lower, df_1.columns)
    df_1 = df_1.drop_duplicates(subset=['retailer_id'], keep="first")
    df_1 = df_1.reset_index(drop=True)
    # print(df_1)
    
    #Remove the data that already exists if found 
    sheet = client.open('[HOURLY] Fintech Runsheet')
    data_instance = sheet.worksheet('Data')
    data_instance_df= pd.DataFrame(data_instance.get('E5:E'))
    # print(data_instance_df)

    if(data_instance_df.empty==False):
        data_instance_df.columns = ['sales_order_id']
        main_data_to_assign = pd.DataFrame(df_1.loc[~df_1['sales_order_id'].isin(data_instance_df['sales_order_id'].astype(int).values)])
        main_data_assigned = assign_data(main_data_to_assign,available_agents_list)
        # print(main_data_assigned)
    else:
        main_data_to_assign = df_1.copy()
        main_data_assigned = assign_data(main_data_to_assign,available_agents_list)
        # print(main_data_assigned)
        
    # agents_df.columns = agents_df.iloc[0]
    # agents_df = agents_df[1:]
    # agents_df.reset_index(drop=True, inplace=True)
    # agents_df
#     grouped_counts = main_data_assigned.groupby('agent_assigned').size().reset_index(name='count')
#     print(grouped_counts)

    #importing data in agents' sheet
    final_data_to_assign = main_data_assigned.copy()
    # final_data_to_assign = final_data_to_assign
    # final_data_to_assign = final_data_to_assign.drop_duplicates(subset='retailer_id', keep="first")
    final_data_to_assign['added_at'] = now.strftime('%Y-%m-%d %H:%M')

    if final_data_to_assign.columns[0] == 'index':
        final_data_to_assign = final_data_to_assign.drop(labels='index', axis=1)
    pass

    final_data_to_assign = final_data_to_assign.astype(str)
    google_sh = client.open('[HOURLY] Fintech Runsheet')
    sheet = google_sh.worksheet('raw_data')
    # sheet.clear()
    sheet.append_rows([final_data_to_assign.columns.values.tolist()] + final_data_to_assign.values.tolist(), value_input_option="USER_ENTERED")
else:
    print("no agents available")


Thursday
0    agent_name agent_id start_time end_time          days_off
0   Fatima Gasi      109          8       18  Satruday, Sunday
1  Doha Rajraji      108          8       18  Saturday, Sunday
9
0    agent_name agent_id start_time end_time          days_off  \
0   Fatima Gasi      109          8       18  Satruday, Sunday   
1  Doha Rajraji      108          8       18  Saturday, Sunday   

0  assignment_start_time  assignment_end_time     agents_days_off assign_data  
0                      8                   18  [Satruday, Sunday]         yes  
1                      8                   18  [Saturday, Sunday]         yes  


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1
7
